In [1]:
import numpy as np
import sys
import math
sys.path.append('/home/user/zhaoy/git/oguz_plantRoot_repo/plant-root-MRI-display/root_extraction/RootStructureExtractor/PyUtils')

In [2]:
import root_graph

In [3]:
# rg = root_graph.RootGraph('/home/user/zhaoy/root_mri/experiments/Roots/lupineSmall_real_step719/root.xml')
rg = root_graph.RootGraph('/home/user/zhaoy/root_mri/experiments/root_extraction_results/rootWeight10/visualized_out_lupine_small_xml_mri/root.xml')


In [6]:
# rg.toXml('/home/user/zhaoy/root_mri/experiments/ls.xml')

In [4]:
print(rg.root)

0: 1.0,[300.0, 235.0, 220.0] 
  0: 9.0,[303.0, 222.0, 208.0] 
  0: 9.0,[305.0, 222.0, 206.0] 
  0: 9.0,[306.0, 211.0, 180.0] 
  0: 8.0,[305.0, 205.0, 169.0] 
  0: 8.0,[299.0, 204.0, 168.0] 
  0: 8.0,[300.0, 204.0, 161.0] 
  0: 9.0,[293.0, 206.0, 156.0] 
  0: 8.0,[288.0, 208.0, 151.0] 
  0: 10.0,[285.0, 203.0, 147.0] 
  0: 6.0,[286.0, 211.0, 146.0] 
  0: 9.0,[283.0, 214.0, 140.0] 
  0: 9.0,[278.0, 215.0, 139.0] 
  0: 10.0,[275.0, 208.0, 133.0] 
  0: 9.0,[264.0, 213.0, 132.0] 
  0: 8.0,[255.0, 208.0, 129.0] 
  0: 9.0,[250.0, 212.0, 129.0] 
  0: 8.0,[244.0, 215.0, 120.0] 
  0: 8.0,[233.0, 216.0, 116.0] 
  0: 8.0,[233.0, 210.0, 109.0] 
  0: 8.0,[231.0, 212.0, 103.0] 
  0: 8.0,[233.0, 200.0, 96.0] 
  0: 11.0,[234.0, 199.0, 91.0] 
  0: 6.0,[218.0, 207.0, 91.0] 
  0: 9.0,[204.0, 205.0, 87.0] 
  0: 9.0,[204.0, 203.0, 81.0] 
  0: 10.0,[203.0, 209.0, 80.0] 
  0: 5.0,[198.0, 207.0, 77.0] 
  0: 8.0,[287.0, 197.0, 155.0] 
  0: 8.0,[293.0, 181.0, 153.0] 
  0: 10.0,[293.0, 161.0, 152.0] 
  0: 10.0,[2

In [5]:
def dfs_tree(root, node_idx, node_list, seg_list):  # node_idx is the index of root 
    child_list = root.children
    for child in child_list:
        # get the coordinates
        x, y, z = child.coor
        child_rad = child.rad
        child_idx = node_list[-1][0]+1
        node_list.append([child_idx, x, y, z])
        # child.branch_id = 0  # only used when there's no branch id for child nodes
        # print(dir(child))  # print the attributes
        branch_id = child.branch_id
        seg_list.append([node_idx, child_idx, branch_id, child_rad])  # node1ID, node2ID, order, brnID, surf [cm2], length [cm], radius [cm], kz [cm4 hPa-1 d-1], kr [cm hPa-1 d-1], emergence time [d], subtype, organtype
        
        if len(child.children)!=0:  # if not leaf node, recursively call this function
            dfs_tree(child, child_idx, node_list, seg_list)  # branch id needs to be updated
            
    return node_list, seg_list

def generate_node_seg_lists(root):
    node_idx = 0
    # root.branch_id = 0
    x, y, z = root.coor
    # rad = root.rad
    node_list = [[node_idx, x, y, z]]
    seg_list = []  # node1ID, node2ID, brnID, radius [cm]
    
    # apply depth first search on the tree structure of the root graph
    node_list, seg_list = dfs_tree(root, node_idx, node_list, seg_list)

    return node_list, seg_list


# def write_dgf(node_list, seg_list, out_path):
#     with open(out_path, 'w') as f:
#         f.write('DGF\nVertex\nparameters 0\n')
#         for l in node_list:
#             f.write('{} {} {}\n'.format(l[1], l[2], l[3]))
#         f.write('#\nSimplex\nparameters 10: node1ID, node2ID, order, brnID, surf [cm2], length [cm], radius [cm], kz [cm4 hPa-1 d-1], kr [cm hPa-1 d-1], emergence time [d], subtype, organtype\n')
#         for l in seg_list:
#             f.write('{} {} {} {} {} {} {} {} {} {} {} {}\n'.format(*l))
#         f.write('#\nBOUNDARYSEGMENTS\n-1 -1\n-1 {}\n#\nBOUNDARYDOMAIN\ndefault 1\n#\n'.format(len(node_list)-1))
    
def write_dgf(filename, nodes, seg, params = np.zeros((0, 0))):
    file = open(filename, "w")  # write file

    nop = params.shape[0]  # number of parameters
    file.write("DGF\n")
    file.write('Vertex\n')
    file.write('parameters 0\n')


    for i in range(0, len(nodes)):
        file.write('{:g} {:g} {:g} \n'.format(nodes[i, 0], nodes[i, 1], nodes[i, 2]))

    file.write('#\n');
    file.write('Simplex\n');
    if nop > 0:
        file.write('parameters {:d}'.format(nop))
        file.write(': node1ID, node2ID, order, brnID, surf [cm2], length [cm], radius [cm], emergence time [d] \n');
    for i in range(0, seg.shape[1]):
        file.write('{:g} {:g}'.format(seg[0,i], seg[1,i]))
        for j in range(0, nop):
            file.write(' {:g}'.format(params[j,i]))
        file.write(' \n')

    # not used...
    file.write('#\nBOUNDARYSEGMENTS\n2 0\n')
    file.write('3 {:g}\n'.format((seg.shape[1])))
    file.write('#\nBOUNDARYDOMAIN\ndefault 1\n#\n')
    file.close()

### use the dfs order to calculate emergence time
def set_LenSur(vertex, prev, nodeID, radius):

    length = np.zeros(len(prev))
    surf = np.zeros(len(prev))
    
    for i in range (0,len(prev)):
        a = vertex[int(prev[i]),:]
        b = vertex[int(nodeID[i]),:]
        length[i] = np.linalg.norm(a-b)
        surf[i] = 2*radius[i]*math.pi*length[i]

    return length, surf

def set_type(branchID, prev):
    """
    Starting from type 1, the successor of branches with type i will be assigned with type i+1
    """

    maxbran = int(max(branchID))
    un_bran = np.unique(branchID)  #unique branches: 0 removed, because the 0 branch only contains the first node
    un_bran = np.asarray(un_bran, int)

    preseg = np.zeros(maxbran)
    prebr = np.zeros(maxbran)
    for i in range(1,maxbran+1):
        res = next(x for x, val in enumerate(branchID) if val == i) #index of every 1st branch segment
        preseg[i-1] = int(prev[res]) #prvious of first br seg
    preseg = np.array(preseg,int)
    prebr = branchID[preseg]  ### the branch id of the parent branch of each branch
    prebr[0] = 0

 
    #find branch types
    branch_type = np.zeros(len(prebr))
    idx = np.where(prebr == 0)
    branch_type[idx[0]] = 1  # the type of main branch is 1

    for i in range(1,len(un_bran)):  # the largest possible branch type is the number of unique branches
        bridx = np.where(branch_type == i) #index of branches with type i
        dumpre = un_bran[bridx[0]]  #num of branches that have type i # To me it's more like the branchID of the first branch of type i
        if len(dumpre)==0:
            break  # once there's no succeeding branches, stop the iteration
        idxpre = np.in1d(prebr, dumpre)  # the child branches whose parent branches are of type i
        branch_type[idxpre] = i+1
      
    #distribute branch types on all segments
    Order = np.zeros(len(branchID))
    Order[np.where(branchID==0)[0]] = 1  # the first segment of the main branch is of type 1
    for i in range(1,maxbran+1):
        idx = np.where(un_bran == i)
        tt = branch_type[idx[0]]  # get the branch type of this branch id
        ix = np.where(branchID==i)
        Order[ix[0]] = tt  # Q: only the 1st seg of the branch is changed? --> no, ix[0] is the result array, ix is a tuple
        
    return Order


def set_age(Order, branchID, prev, length, maxage):
    '''
    Modified algorithm:
    Use the orders (types) of branches to calculate the emergence time of each branch 
      (start time is 0 for the first segment of the main branch, and each branch tip ends at maxage)
    The emergence time of the next order branches (the next level of child branches) will be calculated 
      based on the time tag of the current branches
    '''
    un_typ = np.unique(Order)
    emt = np.zeros(len(Order)) #emergence time 

    for i in range(1, len(un_typ)+1): 
        idx = np.where(Order==i)   #find all segments with this order 
        brtyp = np.unique(branchID[idx[0]]) #find unique branch IDs with this order 

        for j in range(0, len(brtyp)): #iterate through all branchIDs with this order 
            idx1_ = np.where(branchID == brtyp[j])
            idx1 = np.asarray(idx1_[0],int)
            emt[idx1[0]] =  emt[int(prev[idx1[0]])]
            emt[idx1[-1]] = maxage   #tip is set to maxage
            iniage = emt[idx1[0]]
            RL = sum(length[idx1])  # length of this whole branch
            agediff = (maxage-iniage);
            if (agediff!= 0): 
                elong = RL/(maxage-iniage)
            else:
                elong = 0.0001

            for k in range(1, len(idx1)-1): 
                emt[idx1[k]] = emt[idx1[k-1]] + length[idx1[k]]/elong
     
    return emt

def set_max_type(order, maxtyp):
    """
    Set the max order/type: Any order larger than the max order will be converted to the max order
    """
    order[order>maxtyp] = maxtyp
    un_typ = np.unique(order)
            
    return order

In [7]:
def graph_to_dgf(root_graph, max_plant_age, max_type, dgf_output_path):
    node_list, seg_list = generate_node_seg_lists(root_graph.root)
    node_list = np.array(node_list)
    seg_list = np.array(seg_list)
    
    # calculate the length and surface of each segment
    vertex = node_list[:,1:]
    prev, nodeID, branchID, radius = seg_list[:,0], seg_list[:,1], seg_list[:,2], seg_list[:,3]
    length, surf = set_LenSur(vertex, prev, nodeID, radius) 
    
    # calculate the order of each branch segment
    order = set_type(branchID, prev)
    
    # calculate the emergence time
    emt = set_age(order, branchID, prev,length, max_plant_age)  # maximum age of this palnt = 13, shall be given by the user
    
    # set the max type   
    order = set_max_type(order, max_type)
    
    # combine segments and parameters
    seg = (np.stack((prev, nodeID)))
    params = (np.stack((order, branchID, surf, length, radius, emt)))
    
    # write the dgf file:
    write_dgf(dgf_output_path, vertex, seg, params)
    
    print('Dgf file written in path "{}"'.format(dgf_output_path))
    
    return
graph_to_dgf(rg, 13, 2, 'test3.dgf')


Dgf file written in path "test3.dgf"


In [103]:

_, node_idx, node_list, seg_list = initialize(rg.root)
node_list, seg_list = dfs_tree(rg.root, node_idx, node_list, seg_list)
node_list = np.array(node_list)
seg_list = np.array(seg_list)

*******
*******
number of children nodes: 2
*******
number of children nodes: 2
*******
number of children nodes: 2
*******
*******
*******
*******
number of children nodes: 2
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
number of children nodes: 2
*******
number of children nodes: 2
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
number of children nodes: 2
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******
*******


In [104]:
node_list

array([[  0., 300., 235., 220.],
       [  1., 303., 222., 208.],
       [  2., 305., 222., 206.],
       [  3., 306., 211., 180.],
       [  4., 305., 205., 169.],
       [  5., 299., 204., 168.],
       [  6., 300., 204., 161.],
       [  7., 293., 206., 156.],
       [  8., 288., 208., 151.],
       [  9., 285., 203., 147.],
       [ 10., 286., 211., 146.],
       [ 11., 283., 214., 140.],
       [ 12., 278., 215., 139.],
       [ 13., 275., 208., 133.],
       [ 14., 264., 213., 132.],
       [ 15., 255., 208., 129.],
       [ 16., 250., 212., 129.],
       [ 17., 244., 215., 120.],
       [ 18., 233., 216., 116.],
       [ 19., 233., 210., 109.],
       [ 20., 231., 212., 103.],
       [ 21., 233., 200.,  96.],
       [ 22., 234., 199.,  91.],
       [ 23., 218., 207.,  91.],
       [ 24., 204., 205.,  87.],
       [ 25., 204., 203.,  81.],
       [ 26., 203., 209.,  80.],
       [ 27., 198., 207.,  77.],
       [ 28., 287., 197., 155.],
       [ 29., 293., 181., 153.],
       [ 3

In [86]:
# a=np.array([305., 222., 206.])
# b=np.array([306., 211., 180.])
# np.linalg.norm(a-b)

28.24889378365107

In [105]:
seg_list

array([[ 0.,  1.,  1.,  9.],
       [ 1.,  2.,  1.,  9.],
       [ 2.,  3.,  1.,  9.],
       [ 3.,  4.,  1.,  8.],
       [ 4.,  5.,  1.,  8.],
       [ 5.,  6.,  1.,  8.],
       [ 6.,  7.,  1.,  9.],
       [ 7.,  8.,  1.,  8.],
       [ 8.,  9.,  1., 10.],
       [ 9., 10.,  1.,  6.],
       [10., 11.,  1.,  9.],
       [11., 12.,  1.,  9.],
       [12., 13.,  1., 10.],
       [13., 14.,  1.,  9.],
       [14., 15.,  1.,  8.],
       [15., 16.,  1.,  9.],
       [16., 17.,  1.,  8.],
       [17., 18.,  1.,  8.],
       [18., 19.,  1.,  8.],
       [19., 20.,  1.,  8.],
       [20., 21.,  1.,  8.],
       [21., 22.,  1., 11.],
       [22., 23.,  1.,  6.],
       [23., 24.,  1.,  9.],
       [24., 25.,  1.,  9.],
       [25., 26.,  1., 10.],
       [26., 27.,  1.,  5.],
       [ 7., 28.,  5.,  8.],
       [28., 29.,  5.,  8.],
       [29., 30.,  5., 10.],
       [30., 31.,  5., 10.],
       [31., 32.,  5.,  7.],
       [32., 33.,  5.,  7.],
       [ 3., 34.,  3.,  7.],
       [34., 3

In [53]:
# write_dgf(node_list, seg_list, 'test2.dgf')

In [106]:
### use the dfs order to calculate emergence time
def set_LenSur(vertex, prev, nodeID, radius):

    length = np.zeros(len(prev))
    surf = np.zeros(len(prev))
    
    for i in range (0,len(prev)):
        a = vertex[int(prev[i]),:]
        b = vertex[int(nodeID[i]),:]
        length[i] = np.linalg.norm(a-b)
        surf[i] = 2*radius[i]*math.pi*length[i]

    return length, surf

vertex = node_list[:,1:]
prev, nodeID, branchID, radius = seg_list[:,0], seg_list[:,1], seg_list[:,2], seg_list[:,3]
length, surf = set_LenSur(vertex, prev, nodeID, radius)

In [107]:
length

array([17.94435844,  2.82842712, 28.24889378, 12.56980509,  6.164414  ,
        7.07106781,  8.83176087,  7.34846923,  7.07106781,  8.1240384 ,
        7.34846923,  5.19615242,  9.69535971, 12.12435565, 10.72380529,
        6.40312424, 11.22497216, 11.74734012,  9.21954446,  6.63324958,
       14.03566885,  5.19615242, 17.88854382, 14.69693846,  6.32455532,
        6.164414  ,  6.164414  , 10.86278049, 17.20465053, 20.02498439,
        7.87400787, 18.02775638, 19.26136028, 10.63014581, 15.26433752,
       17.3781472 ,  5.38516481,  5.65685425, 11.70469991,  8.94427191,
        7.        , 29.06888371, 21.49418526, 20.46948949, 23.34523506,
       50.80354318, 25.92296279,  6.164414  , 14.49137675, 22.71563338,
       22.44994432,  3.46410162, 58.30951895, 51.2445119 , 21.04756518,
        8.54400375, 14.14213562,  5.38516481, 20.22374842, 12.32882801,
       24.51530134, 63.3245608 , 43.97726685,  7.48331477,  8.94427191,
       11.40175425, 10.81665383, 13.07669683, 20.83266666, 26.60

In [109]:
def set_type(branchID, prev):
    """
    Starting from type 1, the successor of branches with type i will be assigned with type i+1
    """

    maxbran = int(max(branchID))
    un_bran = np.unique(branchID)  #unique branches: 0 removed, because the 0 branch only contains the first node
#     print('np.unique(branchID):', np.unique(branchID))    
    un_bran = np.asarray(un_bran, int)

    preseg = np.zeros(maxbran)
    prebr = np.zeros(maxbran)
    for i in range(1,maxbran+1):
        res = next(x for x, val in enumerate(branchID) if val == i) #index of every 1st branch segment
        preseg[i-1] = int(prev[res]) #prvious of first br seg
    preseg = np.array(preseg,int)
    prebr = branchID[preseg]  ### the branch id of the parent branch of each branch
    prebr[0] = 0

 
    #find branch types
    branch_type = np.zeros(len(prebr))
    idx = np.where(prebr == 0)
    branch_type[idx[0]] = 1  # the type of main branch is 1

    for i in range(1,len(un_bran)):  # the largest possible branch type is the number of unique branches
#         print('len(un_bran):', len(un_bran))
#         print('branch_type:', branch_type)
#         print('un_bran:', un_bran)
        bridx = np.where(branch_type == i) #index of branches with type i
#         print('bridx:', bridx)
        dumpre = un_bran[bridx[0]]  #num of branches that have type i # To me it's more like the branchID of the first branch of type i
#         print('prebr:', prebr)
#         print('dumpre:', dumpre)
#         print()
        if len(dumpre)==0:
            break  # once there's no succeeding branches, stop the iteration
        idxpre = np.in1d(prebr, dumpre)  # the child branches whose parent branches are of type i
        branch_type[idxpre] = i+1
        
    ### idx = np.where(branch_type>maxtyp)  # do this in the end, now the detailed branch orders are needed
    ### branch_type[idx[0]] = maxtyp
      
    #distribute branch types on all segments
    Order = np.zeros(len(branchID))
    Order[np.where(branchID==0)[0]] = 1  # the first segment of the main branch is of type 1
    for i in range(1,maxbran+1):
        idx = np.where(un_bran == i)
        tt = branch_type[idx[0]]  # get the branch type of this branch id
        ix = np.where(branchID==i)
        Order[ix[0]] = tt  # Q: only the 1st seg of the branch is changed? --> no, ix[0] is the result array, ix is a tuple
        
    return Order

o = set_type(branchID, prev)


In [111]:
o

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3.,
       3., 3., 3., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3.])

In [96]:
'''Original code'''

# def set_age(Order, branchID, prev, length,  maxage):

#     un_typ = np.unique(Order)
#     emt = np.zeros(len(Order)) #emergence time 

#     for i in range(1, len(un_typ)+1): 
#         idx = np.where(Order==i)   #find all segments with this order 
#         brtyp = np.unique(branchID[idx[0]]) #find unique branch IDs with this order 

#         for j in range(0, len(brtyp)): #iterate through all branchIDs with this order 
#             # todo: discuss if this algorithm makes sense...
#             # 1. the first branch of a certain type starts and 0 and ends at max age? why not start at the real age?
#             # 2. why the start time of other branches takes the same time as their previous segment?
#             #    Then what's the point of calculation according to different branch type?
#             idx1_ = np.where(branchID == brtyp[j])
#             idx1 = np.asarray(idx1_[0],int)
#             if j == 0: 
#                 emt[idx1[0]] = 0
#             else: 
#                 emt[idx1[0]] =  emt[int(prev[idx1[0]])]

#             emt[idx1[-1]] = maxage   #tip is set to maxage
#             iniage = emt[idx1[0]]
#             RL = sum(length[idx1])  # length of this whole branch
#             agediff = (maxage-iniage);
#             if (agediff!= 0): 
#                 elong = RL/(maxage-iniage)
#             else:
#                 elong = 0.0001

#             for k in range(1, len(idx1)-1): 
#                 emt[idx1[k]] = emt[idx1[k-1]] + length[idx1[k]]/elong
     
#     return emt

# max_plant_age = 13
# emt = set_age(o, branchID, prev,length, max_plant_age)  #maximum age of this palnt = 13, shall be given by the user


In [112]:
'''Modified code'''
def set_age(Order, branchID, prev, length, maxage):
    '''
    Modified algorithm:
    Use the orders (types) of branches to calculate the emergence time of each branch 
      (start time is 0 for the first segment of the main branch, and each branch tip ends at maxage)
    The emergence time of the next order branches (the next level of child branches) will be calculated 
      based on the time tag of the current branches
    '''
    un_typ = np.unique(Order)
    emt = np.zeros(len(Order)) #emergence time 

    for i in range(1, len(un_typ)+1): 
        idx = np.where(Order==i)   #find all segments with this order 
        brtyp = np.unique(branchID[idx[0]]) #find unique branch IDs with this order 

        for j in range(0, len(brtyp)): #iterate through all branchIDs with this order 
            idx1_ = np.where(branchID == brtyp[j])
            idx1 = np.asarray(idx1_[0],int)
            emt[idx1[0]] =  emt[int(prev[idx1[0]])]
            emt[idx1[-1]] = maxage   #tip is set to maxage
            iniage = emt[idx1[0]]
            RL = sum(length[idx1])  # length of this whole branch
            agediff = (maxage-iniage);
            if (agediff!= 0): 
                elong = RL/(maxage-iniage)
            else:
                elong = 0.0001

            for k in range(1, len(idx1)-1): 
                emt[idx1[k]] = emt[idx1[k-1]] + length[idx1[k]]/elong
     
    return emt

max_plant_age = 13
emt = set_age(o, branchID, prev,length, max_plant_age)  #maximum age of this palnt = 13, shall be given by the user


In [113]:
emt

array([ 0.        ,  0.13771917,  1.51318827,  2.12522564,  2.4253776 ,
        2.76967552,  3.19970348,  3.55750838,  3.90180629,  4.29737449,
        4.65517938,  4.90818564,  5.38026316,  5.97061112,  6.49276476,
        6.80453979,  7.35109579,  7.92308645,  8.37199601,  8.69497609,
        9.37838794,  9.63139421, 10.5024067 , 11.21801648, 11.5259659 ,
       11.82611787, 13.        ,  3.55750838,  5.29954717,  7.32715591,
        8.12443029,  9.94981181, 13.        ,  2.12522564,  2.78116111,
        3.52793071,  3.75934073,  4.00242572,  4.50539729,  4.88974841,
        5.19055074,  6.43969188,  7.36333489,  8.24294491,  9.24613079,
       11.42924853, 13.        ,  4.88974841,  6.58604507,  9.24503694,
       11.87292842, 13.        ,  4.50539729,  7.83844821, 13.        ,
        1.51318827,  2.29094066,  2.58709994,  3.69931305,  4.37734186,
        5.72557064,  9.2081301 , 11.62667743, 12.03822531, 13.        ,
        0.13771917,  0.73376948,  1.45435916,  2.60234057,  4.06

In [95]:
emt

array([ 0.        ,  0.13771917,  1.51318827,  2.12522564,  2.4253776 ,
        2.76967552,  3.19970348,  3.55750838,  3.90180629,  4.29737449,
        4.65517938,  4.90818564,  5.38026316,  5.97061112,  6.49276476,
        6.80453979,  7.35109579,  7.92308645,  8.37199601,  8.69497609,
        9.37838794,  9.63139421, 10.5024067 , 11.21801648, 11.5259659 ,
       11.82611787, 13.        ,  3.55750838,  5.29954717,  7.32715591,
        8.12443029,  9.94981181, 13.        ,  2.12522564,  2.78116111,
        3.52793071,  3.75934073,  4.00242572,  4.50539729,  4.88974841,
        5.19055074,  6.43969188,  7.36333489,  8.24294491,  9.24613079,
       11.42924853, 13.        ,  0.        ,  2.71901017,  6.98113372,
       11.19340617, 13.        ,  4.50539729,  7.83844821, 13.        ,
        1.51318827,  2.29094066,  2.58709994,  3.69931305,  4.37734186,
        5.72557064,  9.2081301 , 11.62667743, 12.03822531, 13.        ,
        0.        ,  0.60243235,  1.33073754,  2.49101062,  3.97

In [97]:
emt

array([ 0.        ,  0.13771917,  1.51318827,  2.12522564,  2.4253776 ,
        2.76967552,  3.19970348,  3.55750838,  3.90180629,  4.29737449,
        4.65517938,  4.90818564,  5.38026316,  5.97061112,  6.49276476,
        6.80453979,  7.35109579,  7.92308645,  8.37199601,  8.69497609,
        9.37838794,  9.63139421, 10.5024067 , 11.21801648, 11.5259659 ,
       11.82611787, 13.        ,  0.        ,  2.39836107,  5.18988206,
        6.28753377,  8.80063736, 13.        ,  2.12522564,  2.78116111,
        3.52793071,  3.75934073,  4.00242572,  4.50539729,  4.88974841,
        5.19055074,  6.43969188,  7.36333489,  8.24294491,  9.24613079,
       11.42924853, 13.        ,  0.        ,  2.71901017,  6.98113372,
       11.19340617, 13.        ,  4.50539729,  7.83844821, 13.        ,
        1.51318827,  2.29094066,  2.58709994,  3.69931305,  4.37734186,
        5.72557064,  9.2081301 , 11.62667743, 12.03822531, 13.        ,
        0.13771917,  0.73376948,  1.45435916,  2.60234057,  4.06

In [98]:
o

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3.,
       3., 3., 3., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3.])

In [99]:
dfs_order

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4.,
       4., 5., 5., 5., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 7., 7., 7.,
       7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 8., 8., 8., 8., 8.])

In [100]:
branchID

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 5., 5., 5., 5., 5., 5., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4.,
       4., 7., 7., 7., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 8., 8., 8., 8., 8.])

In [101]:
seg_list

array([[ 0.,  1.,  1.,  9.,  1.],
       [ 1.,  2.,  1.,  9.,  1.],
       [ 2.,  3.,  1.,  9.,  1.],
       [ 3.,  4.,  1.,  8.,  1.],
       [ 4.,  5.,  1.,  8.,  1.],
       [ 5.,  6.,  1.,  8.,  1.],
       [ 6.,  7.,  1.,  9.,  1.],
       [ 7.,  8.,  1.,  8.,  1.],
       [ 8.,  9.,  1., 10.,  1.],
       [ 9., 10.,  1.,  6.,  1.],
       [10., 11.,  1.,  9.,  1.],
       [11., 12.,  1.,  9.,  1.],
       [12., 13.,  1., 10.,  1.],
       [13., 14.,  1.,  9.,  1.],
       [14., 15.,  1.,  8.,  1.],
       [15., 16.,  1.,  9.,  1.],
       [16., 17.,  1.,  8.,  1.],
       [17., 18.,  1.,  8.,  1.],
       [18., 19.,  1.,  8.,  1.],
       [19., 20.,  1.,  8.,  1.],
       [20., 21.,  1.,  8.,  1.],
       [21., 22.,  1., 11.,  1.],
       [22., 23.,  1.,  6.,  1.],
       [23., 24.,  1.,  9.,  1.],
       [24., 25.,  1.,  9.,  1.],
       [25., 26.,  1., 10.,  1.],
       [26., 27.,  1.,  5.,  1.],
       [ 7., 28.,  5.,  8.,  2.],
       [28., 29.,  5.,  8.,  2.],
       [29., 3

In [114]:
o

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3.,
       3., 3., 3., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3.])

In [115]:
emt

array([ 0.        ,  0.13771917,  1.51318827,  2.12522564,  2.4253776 ,
        2.76967552,  3.19970348,  3.55750838,  3.90180629,  4.29737449,
        4.65517938,  4.90818564,  5.38026316,  5.97061112,  6.49276476,
        6.80453979,  7.35109579,  7.92308645,  8.37199601,  8.69497609,
        9.37838794,  9.63139421, 10.5024067 , 11.21801648, 11.5259659 ,
       11.82611787, 13.        ,  3.55750838,  5.29954717,  7.32715591,
        8.12443029,  9.94981181, 13.        ,  2.12522564,  2.78116111,
        3.52793071,  3.75934073,  4.00242572,  4.50539729,  4.88974841,
        5.19055074,  6.43969188,  7.36333489,  8.24294491,  9.24613079,
       11.42924853, 13.        ,  4.88974841,  6.58604507,  9.24503694,
       11.87292842, 13.        ,  4.50539729,  7.83844821, 13.        ,
        1.51318827,  2.29094066,  2.58709994,  3.69931305,  4.37734186,
        5.72557064,  9.2081301 , 11.62667743, 12.03822531, 13.        ,
        0.13771917,  0.73376948,  1.45435916,  2.60234057,  4.06

In [121]:
# def set_max_type(order, emerge_time, maxtyp):
#     """
#     Set the max order/type
#     1. Any order larger than the max order will be converted to the max order
#     2. The emergence time of the earliest emerging branch of each order will be set to 0, 
#        and all the other branches of this order will minus this offset
#     """
#     order[order>maxtyp] = maxtyp
#     un_typ = np.unique(order)

#     for i in range(1, len(un_typ)+1): 
#         idx = np.where(order==i)
#         order_start_time = emerge_time[idx].min()
#         emerge_time[idx] -= order_start_time
            
#     return order, emerge_time
# o2, emt2 = set_max_type(o, emt, 3)

def set_max_type(order, maxtyp):
    """
    Set the max order/type
    1. Any order larger than the max order will be converted to the max order
    2. The emergence time of the earliest emerging branch of each order will be set to 0, 
       and all the other branches of this order will minus this offset
    """
    order[order>maxtyp] = maxtyp
    un_typ = np.unique(order)
            
    return order
o2 = set_max_type(o, 2)


In [118]:
emt2

array([ 0.        ,  0.13771917,  1.51318827,  2.12522564,  2.4253776 ,
        2.76967552,  3.19970348,  3.55750838,  3.90180629,  4.29737449,
        4.65517938,  4.90818564,  5.38026316,  5.97061112,  6.49276476,
        6.80453979,  7.35109579,  7.92308645,  8.37199601,  8.69497609,
        9.37838794,  9.63139421, 10.5024067 , 11.21801648, 11.5259659 ,
       11.82611787, 13.        ,  3.41978921,  5.16182801,  7.18943674,
        7.98671113,  9.81209264, 12.86228083,  1.98750647,  2.64344194,
        3.39021154,  3.62162156,  3.86470655,  4.36767813,  4.75202925,
        5.05283158,  6.30197272,  7.22561572,  8.10522574,  9.10841162,
       11.29152936, 12.86228083,  2.28740785,  3.9837045 ,  6.64269638,
        9.27058786, 10.39765943,  1.90305673,  5.23610765, 10.39765943,
        1.3754691 ,  2.15322149,  2.44938078,  3.56159388,  4.2396227 ,
        5.58785147,  9.07041094, 11.48895826, 11.90050614, 12.86228083,
        0.        ,  0.59605031,  1.31664   ,  2.4646214 ,  3.93

In [120]:
o2

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3.,
       3., 3., 3., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3.])

In [122]:
o2

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])